In [1]:
import torch
import numpy as np

In [2]:
# 方法一
x = torch.randn(3, 4, requires_grad=True) # requires_grad可以对当前x进行求导
x

tensor([[-1.2367,  0.0682,  1.0653,  0.8201],
        [ 0.8362, -1.4021,  1.8466, -0.1803],
        [-0.1978, -0.2123, -1.3564, -0.6358]], requires_grad=True)

In [3]:
# 方法二
x = torch.randn(3, 4)
x.requires_grad = True
x

tensor([[-0.1289,  0.3629,  0.8613,  0.0067],
        [-0.6045,  1.0934, -1.1995,  1.0265],
        [-1.7146,  1.5572,  0.2270, -0.0674]], requires_grad=True)

In [4]:
b = torch.randn(3, 4, requires_grad=True)
b

tensor([[-0.2225, -0.0142,  1.1115,  0.8699],
        [ 0.3337, -0.0066,  0.1678, -0.2839],
        [ 1.3973, -0.1481, -0.0161, -0.5069]], requires_grad=True)

In [5]:
t = x + b
t

tensor([[-0.3514,  0.3487,  1.9729,  0.8766],
        [-0.2708,  1.0868, -1.0316,  0.7426],
        [-0.3173,  1.4091,  0.2109, -0.5743]], grad_fn=<AddBackward0>)

In [6]:
y = t.sum()
y

tensor(4.1022, grad_fn=<SumBackward0>)

In [7]:
y.backward()

In [8]:
b.grad # 对b求导

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

**requires_grad参数默认为True**

In [9]:
# 计算流程
x = torch.rand(1)
b = torch.rand(1, requires_grad = True)
w = torch.rand(1, requires_grad = True)
y = w * b
z = y + b

In [10]:
# 反向传播计算
z.backward(retain_graph=True)

# 线性回归案例

### 构造一组输入数据和其对应的标签y


In [11]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)
x_train.shape

(11, 1)

In [12]:
y_values = [2 * i for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [16]:
import torch
import torch.nn as nn

## 线性回归模型
- 其实线性回归就是一个不加激活函数的全连接层

In [20]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.linear(x)
        return out

In [22]:
input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)

指定好参数和损失函数

In [24]:
epochs = 1000 # 训练次数
learning_rate = 0.01 # 学习率
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate) # 指定优化器
critertion = nn.MSELoss() # 设定损失函数

训练模型

In [26]:
for epoch in range(epochs):
    epoch += 1
    
    # 将np转换为tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)
    
    # 梯度要清零每一次迭代
    optimizer.zero_grad()
    
    # 向前传播
    outputs = model(inputs)
    
    # 计算损失
    loss = critertion(outputs, labels)
    
    # 反向传播
    loss.backward()
    
    # 更新权重参数
    optimizer.step()
    if epoch % 50 == 0:
        print("epoch {}, loss {}".format(epoch, loss.item()))

epoch 50, loss 6.50688676273603e-08
epoch 100, loss 3.709802598450551e-08
epoch 150, loss 2.115869612850929e-08
epoch 200, loss 1.207325439622764e-08
epoch 250, loss 6.9102465971582205e-09
epoch 300, loss 3.95611854386857e-09
epoch 350, loss 2.27043606138011e-09
epoch 400, loss 1.2890045697488972e-09
epoch 450, loss 7.367341736674859e-10
epoch 500, loss 4.3029951823925217e-10
epoch 550, loss 2.493215633947443e-10
epoch 600, loss 1.429334012259531e-10
epoch 650, loss 8.555636754614682e-11
epoch 700, loss 4.958170332636236e-11
epoch 750, loss 3.125026279415799e-11
epoch 800, loss 1.9056083100377208e-11
epoch 850, loss 1.179553676955436e-11
epoch 900, loss 7.700250159725641e-12
epoch 950, loss 6.19150173838956e-12
epoch 1000, loss 5.177509896509003e-12


## 测试模型预测结果

In [27]:
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[-3.7852265e-06],
       [ 1.9999969e+00],
       [ 3.9999976e+00],
       [ 5.9999981e+00],
       [ 7.9999990e+00],
       [ 1.0000000e+01],
       [ 1.2000000e+01],
       [ 1.4000001e+01],
       [ 1.6000002e+01],
       [ 1.8000002e+01],
       [ 2.0000004e+01]], dtype=float32)

## 模型的保存与读取

In [28]:
torch.save(model.state_dict(), "model.pkl")

In [29]:
model.load_state_dict(torch.load("model.pkl"))

<All keys matched successfully>

## 使用GPU进行训练